In [1]:
!nvidia-smi
!pip install x-transformers

Mon Dec 13 21:52:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone https://github.com/wingedsheep/music-generation-toolbox

Cloning into 'music-generation-toolbox'...
remote: Enumerating objects: 1441, done.
remote: Counting objects: 100% (533/533), done.
remote: Compressing objects: 100% (341/341), done.
remote: Total 1441 (delta 367), reused 340 (delta 188), pack-reused 908
Receiving objects: 100% (1441/1441), 8.54 MiB | 23.31 MiB/s, done.
Resolving deltas: 100% (941/941), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pretty_midi
!pip install sequitur

     |████████████████████████████████| 5.6 MB 5.4 MB/s 
     |████████████████████████████████| 51 kB 6.8 MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591953 sha256=cf19fa4128673cd6c5b53f7fae89e509834259e34aa35a3efce9c03d00934f5b
  Stored in directory: /root/.cache/pip/wheels/ad/74/7c/a06473ca8dcb63efb98c1e67667ce39d52100f837835ea18fa
Successfully built pretty-midi
  Created wheel for sequitur: filename=sequitur-1.2.4-py3-none-any.whl size=9627 sha256=af7f5550e0dffba305b25d56d6d1b7ad3c02f358a5962d85d0c72d4e44ad006d
  Stored in directory: /root/.cache/pip/wheels/a6/72/17/d63e70847cd9e9ee2d14d3a023ca2569a3be8330ac20918ea6
Successfully built sequitur


In [4]:
%cd music-generation-toolbox/

/content/music-generation-toolbox


In [5]:
!git checkout feature/encoded_beats

Branch 'feature/encoded_beats' set up to track remote branch 'feature/encoded_beats' from 'origin'.
Switched to a new branch 'feature/encoded_beats'


In [7]:
from pretty_midi import pretty_midi

from mgt.datamanagers.encoded_beats.beat_data_extractor import BeatDataExtractor
from mgt.models.encoded_beats_model import EncodedBeatsModel

In [8]:
data_extractor = BeatDataExtractor()
midi_path = '/content/californication_4_tracks.mid'
midi_data = pretty_midi.PrettyMIDI(midi_path)
beats_matrices = data_extractor.extract_beats(midi_data)

/usr/local/lib/python3.7/dist-packages/pretty_midi/pretty_midi.py:101: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)


Created a matrix with shape (508, 2048)


In [23]:
restored_beats_matrices = beats_matrices.reshape(len(beats_matrices), 4, 4, 128)

In [45]:
notes = []

for beat_index, beat in enumerate(restored_beats_matrices):
  for sub_beat_index, sub_beat in enumerate(beat):
    for instrument_index, instrument in enumerate(sub_beat):
      for pitch_index, pitch in enumerate(instrument):
        if pitch == 1:
          notes.append({'instrument': instrument_index, 'pitch': pitch_index, 'time': beat_index * 4 + sub_beat_index, 'type': 'on'})
        elif pitch == 2:
          notes.append({'instrument': instrument_index, 'pitch': pitch_index, 'time': beat_index * 4 + sub_beat_index, 'type': 'off'})
        
sorted_notes = sorted(notes, key=lambda x: (x['instrument'], x['time']))

midi = pretty_midi.PrettyMIDI()

instruments = [
      pretty_midi.Instrument(program=27),
      pretty_midi.Instrument(program=70),
      pretty_midi.Instrument(program=33),
      pretty_midi.Instrument(program=0, is_drum=True)         
]

for instrument in instruments:
  midi.instruments.append(instrument)

for note in sorted_notes:
  instrument = instruments[note['instrument']]
  end_time = note['time'] * (0.5 / 4) + (0.5 * 4)
  midi_note = pretty_midi.Note(
      velocity=64,
      pitch=note['pitch'],
      start=note['time'] * (0.5 / 4),
      end=end_time
  )
  instrument.notes.append(midi_note)

midi.write('/content/test_output.midi')


In [60]:
# encoded_beats_model = EncodedBeatsModel()
# encoded_beats_model.auto_encoder.train(beats_matrices, 2000)
encoded_beats_model.train([beats_matrices], epochs=50, batch_size=12)
result = encoded_beats_model.generate(output_length=200)

Training epoch 1.
Processed 1 / 1 with loss 0.01632019877433777.
Loss after epoch 1 is 0.01632019877433777. Running time: 1.5952258110046387
Training epoch 2.
Processed 1 / 1 with loss 0.01632542908191681.
Loss after epoch 2 is 0.01632542908191681. Running time: 3.1915838718414307
Training epoch 3.
Processed 1 / 1 with loss 0.016117189079523087.
Loss after epoch 3 is 0.016117189079523087. Running time: 4.788671255111694
Training epoch 4.
Processed 1 / 1 with loss 0.016155365854501724.
Loss after epoch 4 is 0.016155365854501724. Running time: 6.387181758880615
Training epoch 5.
Processed 1 / 1 with loss 0.016010625287890434.
Loss after epoch 5 is 0.016010625287890434. Running time: 7.984698057174683
Training epoch 6.
Processed 1 / 1 with loss 0.01563827134668827.
Loss after epoch 6 is 0.01563827134668827. Running time: 9.583722829818726
Training epoch 7.
Processed 1 / 1 with loss 0.015725623816251755.
Loss after epoch 7 is 0.015725623816251755. Running time: 11.178402423858643
Training 

In [12]:
import numpy as np

print(np.array(result).shape)

(20, 2048)


In [61]:
reshaped = np.array(result).reshape(len(result), len(data_extractor.instruments), data_extractor.beat_resolution, 128)
print(reshaped.shape)

(200, 4, 4, 128)


In [62]:
notes = []

for beat_index, beat in enumerate(reshaped):
  for sub_beat_index, sub_beat in enumerate(beat):
    for instrument_index, instrument in enumerate(sub_beat):
      for pitch_index, pitch in enumerate(instrument):
        if pitch == 1:
          notes.append({'instrument': instrument_index, 'pitch': pitch_index, 'time': beat_index * 4 + sub_beat_index, 'type': 'on'})
        elif pitch == 2:
          notes.append({'instrument': instrument_index, 'pitch': pitch_index, 'time': beat_index * 4 + sub_beat_index, 'type': 'off'})
        
sorted_notes = sorted(notes, key=lambda x: (x['instrument'], x['time']))

midi = pretty_midi.PrettyMIDI()

instruments = [
      pretty_midi.Instrument(program=27),
      pretty_midi.Instrument(program=70),
      pretty_midi.Instrument(program=33),
      pretty_midi.Instrument(program=0, is_drum=True)         
]

for instrument in instruments:
  midi.instruments.append(instrument)

for note in sorted_notes:
  instrument = instruments[note['instrument']]
  end_time = note['time'] * (0.5 / 4) + (0.5 * 4)
  midi_note = pretty_midi.Note(
      velocity=64,
      pitch=note['pitch'],
      start=note['time'] * (0.5 / 4),
      end=end_time
  )
  instrument.notes.append(midi_note)

midi.write('/content/test_output.midi')

In [35]:
midi_path2 = '/content/test.mid'
midi_data2 = pretty_midi.PrettyMIDI(midi_path2)

/usr/local/lib/python3.7/dist-packages/pretty_midi/pretty_midi.py:101: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)


In [36]:
for instrument in midi_data2.instruments:
  for note in instrument.notes:
    print(note)

Note(start=0.000000, end=0.473958, pitch=60, velocity=80)
Note(start=0.500000, end=0.973958, pitch=60, velocity=80)
Note(start=1.000000, end=1.473958, pitch=60, velocity=80)
Note(start=1.500000, end=1.736458, pitch=60, velocity=80)
Note(start=1.750000, end=1.986458, pitch=60, velocity=80)


In [ ]:
data_manager = CompoundWordDataManager()

In [ ]:
model = CompoundWordTransformerModel.load_checkpoint("/content/drive/MyDrive/Music generation toolbox/Generated/2021-08-20 pop909 compound/model_3")

In [ ]:
model = CompoundWordTransformerModel(
    max_sequence_length=512,
    depth=12,
    heads=6
)

In [ ]:
from mgt.datamanagers.data_helper import DataHelper

data = []

data_paths = [
  '/content/drive/MyDrive/Music generation toolbox/Input/Pop 909/pop909_compound_transposed2'
]

for data_path in data_paths:
  new_data = DataHelper.load(data_path)
  data.extend(new_data)

In [ ]:
import random

for i in range(0, 200):
  model.train(x_train=[data[2]], epochs=10, stop_loss=0.001, batch_size=10)
  output = model.generate(1000)
  midi = data_manager.to_midi(output)
  midi.save(f"/content/sample_{i}.midi")

Training epoch 1.
Processed 20 / 100 with loss 1.9322932243347168.
Processed 40 / 100 with loss 1.0992321908473968.
Processed 60 / 100 with loss 0.9666453301906586.
Processed 80 / 100 with loss 0.8689577668905258.
Processed 100 / 100 with loss 0.736160370707512.
Loss after epoch 1 is 1.1206577765941619. Running time: 24.214993238449097
Training epoch 2.
Processed 20 / 100 with loss 0.5804403841495513.
Processed 40 / 100 with loss 0.43773161470890043.
Processed 60 / 100 with loss 0.31507809907197953.
Processed 80 / 100 with loss 0.22222747877240182.
Processed 100 / 100 with loss 0.1487072642892599.
Loss after epoch 2 is 0.34083696819841863. Running time: 48.34664607048035
Training epoch 3.
Processed 20 / 100 with loss 0.09320541806519031.
Processed 40 / 100 with loss 0.06285532899200916.
Processed 60 / 100 with loss 0.043929393589496615.
Processed 80 / 100 with loss 0.02925686500966549.
Processed 100 / 100 with loss 0.022498731687664985.
Loss after epoch 3 is 0.05034914746880531. Runnin

KeyboardInterrupt: ignored

In [ ]:
output = model.generate(1000)
midi = data_manager.to_midi(output)
midi.save(f"/content/sample_{i}.midi")

Generating a new song with 1000 characters.
------ initiate ------
------ generate ------
Writing midi to /content/sample_0.midi
/content/sample_0.midi
